<a href="https://colab.research.google.com/github/manoj-b-22/IDS-in-IoV/blob/main/code/CNN-intra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, Dense, AveragePooling1D
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv('/content/drive/MyDrive/intravehicle.csv')

In [6]:
y_data = data.pop('4')
x_data = data

In [7]:
train_x,test_x,train_y,test_y = train_test_split(x_data, y_data, test_size=0.2, random_state=0, shuffle=True) 

In [8]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [9]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [10]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.transform(test_y)

In [11]:
model = Sequential()
model.add(Conv1D(activation="sigmoid", input_shape=(len(x_data.columns),1), kernel_size=2, filters=100))
model.add(AveragePooling1D(pool_size=2))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 100)            300       
                                                                 
 average_pooling1d (AverageP  (None, 1, 100)           0         
 ooling1D)                                                       
                                                                 
 dense (Dense)               (None, 1, 4)              404       
                                                                 
Total params: 704
Trainable params: 704
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(train_x, train_y, batch_size=32, epochs=6)

Epoch 1/6
414237/414237 [==============================] - 1151s 3ms/step - loss: 0.2170 - accuracy: 0.7970
Epoch 2/6
414237/414237 [==============================] - 1097s 3ms/step - loss: 0.1876 - accuracy: 0.7937
Epoch 3/6
414237/414237 [==============================] - 1119s 3ms/step - loss: 0.1800 - accuracy: 0.7959
Epoch 4/6
414237/414237 [==============================] - 1112s 3ms/step - loss: 0.1750 - accuracy: 0.7959
Epoch 5/6
414237/414237 [==============================] - 1120s 3ms/step - loss: 0.1710 - accuracy: 0.7956
Epoch 6/6
414237/414237 [==============================] - 1042s 3ms/step - loss: 0.1669 - accuracy: 0.7956


In [13]:
pred = model.predict(test_x, verbose=1)
pred_y = np.argmax(pred, axis=2)

103560/103560 [==============================] - 176s 2ms/step


In [25]:
confusion_matrix(test_y,pred_y)

array([[  17849,       0,   99878,       0],
       [      0,   57305,   40897,       0],
       [  10815,   12271, 2824675,     149],
       [      0,       0,  119226,  130830]])

In [23]:
report = classification_report(test_y, pred_y)
print(report)

              precision    recall  f1-score   support

           0       0.62      0.15      0.24    117727
           1       0.82      0.58      0.68     98202
           2       0.92      0.99      0.95   2847910
           3       1.00      0.52      0.69    250056

    accuracy                           0.91   3313895
   macro avg       0.84      0.56      0.64   3313895
weighted avg       0.91      0.91      0.90   3313895

